<a href="https://colab.research.google.com/github/Ellen-Tuane/Analise-Cenario-Ensino-Superior/blob/main/Dados_Cota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise Preliminar dos Dados de 2020 do Ensino Superior

Fonte dos Dados: https://www.gov.br/inep/pt-br/areas-de-atuacao/pesquisas-estatisticas-e-indicadores/censo-da-educacao-superior



# Mount drive, import libraries, unzip files

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [69]:
import pandas as pd
import zipfile

In [70]:
import plotly.express as px
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go

In [ ]:
with zipfile.ZipFile("/content/drive/MyDrive/microdados_censo_da_educacao_superior_2020.zip") as z:
    print(*z.namelist(), sep='\n')

In [4]:
with zipfile.ZipFile("/content/drive/MyDrive/microdados_censo_da_educacao_superior_2020.zip") as z:
    with z.open('Microdados do Censo da Educaç╞o Superior 2020/dados/MICRODADOS_CADASTRO_CURSOS_2020.CSV') as f:
        df = pd.read_csv(f, sep=';', encoding='ISO-8859-1')

# Números Totais de Ingressantes, Matriculados e Concluintes

In [7]:
df_ingressantes = df.loc[:,['NU_ANO_CENSO','NO_REGIAO','NO_UF','TP_CATEGORIA_ADMINISTRATIVA','TP_REDE','QT_ING']]
print('Quantidade Total de Ingressantes no Ensino Superior 2020: ', df.QT_ING.sum())
print('Quantidade Total de Matriculados no Ensino Superior 2020: ', df.QT_MAT.sum())
print('Quantidade Total de Concluintes no Ensino Superior 2020: ', df.QT_CONC.sum())

Quantidade Total de Ingressantes no Ensino Superior 2020:  3765669
Quantidade Total de Matriculados no Ensino Superior 2020:  8680945
Quantidade Total de Concluintes no Ensino Superior 2020:  1278755


# Números Totais de Ingressantes por Região




In [54]:
df_qtd_ing_regiao = df.groupby(['NO_REGIAO']).agg(QTD_INGRESSANTES = ('QT_ING','sum')).reset_index()
#df_qtd_ing_regiao

In [72]:
def plot(x, y, title, x_title,y_title):

    x=x
    y=y

    fig_ing_regiao = go.Figure(data=[go.Bar(x=x, y=y, text=y, textposition='auto')])
    fig_ing_regiao.update_layout(
        title={'text':title, 
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            xaxis_title=x_title,
            yaxis_title=y_title, 
            uniformtext_minsize=15, uniformtext_mode='hide',
            font=dict(
                family="Courier New, monospace",
                size=18)
            )

    return fig_ing_regiao.show()

In [73]:
plot(df_qtd_ing_regiao['NO_REGIAO'],df_qtd_ing_regiao['QTD_INGRESSANTES'], "Número Totais de Ingressantes por Região","Região","Quantidade de Ingressantes")

# Número Totais de Ingressantes por Região: 

*   1 Publicas
*   2 Privadas



In [106]:
qtd_ing_regiao_tp_rede = df.groupby(['NO_REGIAO','TP_REDE']).agg(QT_ING = ('QT_ING','sum')).reset_index()

In [107]:
fig_ing_regiao_tp_rede = px.bar(qtd_ing_regiao_tp_rede, x='NO_REGIAO', y='QT_ING',barmode='group',text_auto=True,color='TP_REDE', labels={'QT_ING':'Quantidade de Ingressantes'})
fig_ing_regiao_tp_rede.update_layout(
        title={'text':'Número Totais de Ingressantes por Região', 
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            xaxis_title='Região',
            yaxis_title='Quantidade de Ingressantes', 
            uniformtext_minsize=15, uniformtext_mode='hide',
            font=dict(
                family="Courier New, monospace",
                size=18)
            )

fig_ing_regiao_tp_rede.show()

# Distibuição por municipio

In [ ]:
df_qtd_ing_municipio= df.groupby(['CO_MUNICIPIO','NO_MUNICIPIO','TP_REDE']).agg(QTD_ING = ('QT_ING','sum')).reset_index()
df_qtd_ing_municipio

In [ ]:
df_municipios = pd.read_csv('/content/drive/MyDrive/Analise Cota Ensino Superior/municipios.csv')
df_municipios

In [ ]:
df_qtd_ing_municipio_lat_long = pd.merge(df_qtd_ing_municipio, df_municipios,how='left', left_on=['CO_MUNICIPIO'],right_on=['codigo_ibge'])
df_qtd_ing_municipio_lat_long

In [ ]:
df_ddd = df_qtd_ing_municipio_lat_long.groupby('ddd').agg(QTD_INGRESSANTES = ('QTD_ING','sum')).reset_index()
df_ddd

In [ ]:
df_qtd_ing_municipio_lat_long.plot.scatter(y='latitude', x='longitude', c=df_qtd_ing_municipio_lat_long['QTD_ING'], cmap='RdYlGn', figsize=(10,8))


#Quantidade por Cor

In [178]:
def plot_cor(df,x,y,title):
    fig_ing_regiao_cor = px.bar(df, x=x, 
                                y=y,
                                barmode='group',text_auto='.2s')
    fig_ing_regiao_cor.update_layout(
            title={'text':title, 
                'y':0.99,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                xaxis_title='Região',
                yaxis_title='Quantidade de Ingressantes', 
                uniformtext_minsize=15,
                font=dict(
                    family="Courier New, monospace",
                    size=15)
                )

    fig_ing_regiao_cor.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)

    return fig_ing_regiao_cor.show()

In [185]:
df_qtd_ing_regiao_cor = df.groupby(['NO_REGIAO']).agg(QT_ING_BRANCA = ('QT_ING_BRANCA','sum'),
                                                            QT_ING_PRETA = ('QT_ING_PRETA','sum'),
                                                            QT_ING_PARDA = ('QT_ING_PARDA', 'sum'),
                                                            QT_ING_AMARELA = ('QT_ING_AMARELA', 'sum'),
                                                            QT_ING_INDIGENA = ('QT_ING_INDIGENA', 'sum'),
                                                            QT_ING_CORND = ('QT_ING_CORND', 'sum'),
                                                            ).reset_index()
#df_qtd_ing_regiao_cor

In [180]:
plot_cor(df_qtd_ing_regiao_cor, 'NO_REGIAO',['QT_ING_BRANCA','QT_ING_PRETA', 'QT_ING_PARDA','QT_ING_AMARELA','QT_ING_INDIGENA','QT_ING_CORND'],
         'Número Totais de Ingressantes por Região')

In [ ]:
df_total_ing_regiao_cor = df_qtd_ing_regiao_cor.iloc[1:].sum().reset_index()
df_total_ing_regiao_cor.drop(index=df_total_ing_regiao_cor.index[0], axis=0, inplace=True)
df_total_ing_regiao_cor = df_total_ing_regiao_cor.rename({'index': 'COR', '0': 'QT_INT'}, axis=1)
df_total_ing_regiao_cor

In [193]:
df_qtd_mat_regiao_cor = df.groupby(['NO_REGIAO']).agg(QT_MAT_BRANCA = ('QT_MAT_BRANCA','sum'),
                                                            QT_MAT_PRETA = ('QT_MAT_PRETA','sum'),
                                                            QT_MAT_PARDA = ('QT_MAT_PARDA', 'sum'),
                                                            QT_MAT_AMARELA = ('QT_MAT_AMARELA', 'sum'),
                                                            QT_MAT_INDIGENA = ('QT_MAT_INDIGENA', 'sum'),
                                                            QT_MAT_CORND = ('QT_MAT_CORND', 'sum'),
                                                            ).reset_index()
#df_qtd_mat_regiao_cor

In [195]:
plot_cor(df_qtd_mat_regiao_cor, 'NO_REGIAO', ['QT_MAT_BRANCA','QT_MAT_PRETA', 'QT_MAT_PARDA','QT_MAT_AMARELA','QT_MAT_INDIGENA','QT_MAT_CORND'],
         'Número Totais de Matriculados por Região')

In [187]:
df_qtd_conc_regiao_cor = df.groupby(['NO_REGIAO']).agg(QT_CONC_BRANCA = ('QT_CONC_BRANCA','sum'),
                                                            QT_CONC_PRETA = ('QT_CONC_PRETA','sum'),
                                                            QT_CONC_PARDA = ('QT_CONC_PARDA', 'sum'),
                                                            QT_CONC_AMARELA = ('QT_CONC_AMARELA', 'sum'),
                                                            QT_CONC_INDIGENA = ('QT_CONC_INDIGENA', 'sum'),
                                                            QT_CONC_CORND = ('QT_CONC_CORND', 'sum'),
                                                            ).reset_index()
#df_qtd_conc_regiao_cor

In [188]:
plot_cor(df_qtd_conc_regiao_cor, 'NO_REGIAO', ['QT_CONC_BRANCA','QT_CONC_PRETA', 'QT_CONC_PARDA','QT_CONC_AMARELA','QT_CONC_INDIGENA','QT_CONC_CORND'],
         'Número Totais de Concluintes por Região')

In [189]:
df_qtd_ing_regiao_cor_tipo = df.groupby(['NO_REGIAO','TP_REDE']).agg(QT_ING_BRANCA = ('QT_ING_BRANCA','sum'),
                                                            QT_ING_PRETA = ('QT_ING_PRETA','sum'),
                                                            QT_ING_PARDA = ('QT_ING_PARDA', 'sum'),
                                                            QT_ING_AMARELA = ('QT_ING_AMARELA', 'sum'),
                                                            QT_ING_INDIGENA = ('QT_ING_INDIGENA', 'sum'),
                                                            QT_ING_CORND = ('QT_ING_CORND', 'sum'),
                                                            ).reset_index()
df_qtd_ing_regiao_cor_tipo

,NO_REGIAO,TP_REDE,QT_ING_BRANCA,QT_ING_PRETA,QT_ING_PARDA,QT_ING_AMARELA,QT_ING_INDIGENA,QT_ING_CORND
0,Centro-Oeste,1,15722,5050,18525,692,622,5414
1,Centro-Oeste,2,83614,17128,106251,6896,1183,83944
2,Nordeste,1,36111,17564,65075,1680,687,20262
3,Nordeste,2,133090,40494,235253,8354,2238,159059
4,Norte,1,7675,4207,25522,531,763,1497
5,Norte,2,51290,10043,126379,3310,1762,71610
6,Sudeste,1,102240,19978,55337,3110,459,28578
7,Sudeste,2,628863,105889,365352,20289,4181,392865
8,Sul,1,61813,4940,13672,1186,549,7664
9,Sul,2,334771,19564,64269,4803,1018,153060


# Deficientes

In [ ]:
df_qtd_deficientes = df.groupby(['NO_REGIAO']).agg(QT_ALUNO_DEFICIENTE=('QT_ALUNO_DEFICIENTE','sum')).reset_index()
df_qtd_deficientes

,NO_REGIAO,QT_ALUNO_DEFICIENTE
0,Centro-Oeste,8030
1,Nordeste,15169
2,Norte,8844
3,Sudeste,30350
4,Sul,14246


In [ ]:
df_qtd_deficientes_detalhado = df.groupby(['NO_REGIAO']).agg(QT_ING_DEFICIENTE=('QT_ING_DEFICIENTE','sum'),
                                                   QT_MAT_DEFICIENTE =('QT_MAT_DEFICIENTE','sum'),
                                                  QT_CONC_DEFICIENTE = ('QT_CONC_DEFICIENTE','sum')).reset_index()
df_qtd_deficientes_detalhado

,NO_REGIAO,QT_ING_DEFICIENTE,QT_MAT_DEFICIENTE,QT_CONC_DEFICIENTE
0,Centro-Oeste,2171,5719,851
1,Nordeste,3779,11206,1129
2,Norte,2677,6191,708
3,Sudeste,9296,22012,3480
4,Sul,4129,10732,1434


In [ ]:
df_qtd_deficientes_detalhado_tipo = df.groupby(['NO_REGIAO','TP_REDE']).agg(QT_ING_DEFICIENTE=('QT_ING_DEFICIENTE','sum'),
                                                   QT_MAT_DEFICIENTE =('QT_MAT_DEFICIENTE','sum'),
                                                  QT_CONC_DEFICIENTE = ('QT_CONC_DEFICIENTE','sum')).reset_index()
df_qtd_deficientes_detalhado_tipo

,NO_REGIAO,TP_REDE,QT_ING_DEFICIENTE,QT_MAT_DEFICIENTE,QT_CONC_DEFICIENTE
0,Centro-Oeste,1,590,1799,110
1,Centro-Oeste,2,1581,3920,741
2,Nordeste,1,1709,6441,380
3,Nordeste,2,2070,4765,749
4,Norte,1,783,2016,116
5,Norte,2,1894,4175,592
6,Sudeste,1,1894,5841,388
7,Sudeste,2,7402,16171,3092
8,Sul,1,870,3148,267
9,Sul,2,3259,7584,1167


#Tipo de ingresso

In [ ]:
df_qtd_ing_regiao_reserva_vagas = df.groupby(['NO_REGIAO','TP_REDE']).agg(QT_ING_RESERVA_VAGA = ('QT_ING_RESERVA_VAGA','sum'),
                                                            QT_ING_RVREDEPUBLICA = ('QT_ING_RVREDEPUBLICA','sum'),
                                                            QT_ING_RVETNICO = ('QT_ING_RVETNICO', 'sum'),
                                                            QT_ING_RVPDEF = ('QT_ING_RVPDEF', 'sum'),
                                                            QT_ING_RVSOCIAL_RF = ('QT_ING_RVSOCIAL_RF', 'sum'),
                                                            QT_ING_RVOUTROS = ('QT_ING_RVOUTROS', 'sum'),
                                                            ).reset_index()
df_qtd_ing_regiao_reserva_vagas

,NO_REGIAO,TP_REDE,QT_ING_RESERVA_VAGA,QT_ING_RVREDEPUBLICA,QT_ING_RVETNICO,QT_ING_RVPDEF,QT_ING_RVSOCIAL_RF,QT_ING_RVOUTROS
0,Centro-Oeste,1,14971,13236,7815,371,5564,407
1,Centro-Oeste,2,261,0,0,1,17,243
2,Nordeste,1,48996,45110,29184,936,18040,1320
3,Nordeste,2,488,216,2,0,4,376
4,Norte,1,13131,12561,9170,395,4940,108
5,Norte,2,27,18,3,8,1,0
6,Sudeste,1,49794,39142,23820,725,20338,911
7,Sudeste,2,156,71,11,3,93,3
8,Sul,1,27763,25558,5151,346,8517,474
9,Sul,2,15,10,3,2,4,0


In [ ]:
df_qtd_conc_regiao_reserva_vagas = df.groupby(['NO_REGIAO','TP_REDE']).agg(QT_CONC_RESERVA_VAGA = ('QT_CONC_RESERVA_VAGA','sum'),
                                                            QT_CONC_RVREDEPUBLICA = ('QT_CONC_RVREDEPUBLICA','sum'),
                                                            QT_CONC_RVETNICO = ('QT_CONC_RVETNICO', 'sum'),
                                                            QT_CONC_RVPDEF = ('QT_CONC_RVPDEF', 'sum'),
                                                            QT_CONC_RVSOCIAL_RF = ('QT_CONC_RVSOCIAL_RF', 'sum'),
                                                            QT_CONC_RVOUTROS = ('QT_CONC_RVOUTROS', 'sum'),
                                                            ).reset_index()
df_qtd_conc_regiao_reserva_vagas

,NO_REGIAO,TP_REDE,QT_CONC_RESERVA_VAGA,QT_CONC_RVREDEPUBLICA,QT_CONC_RVETNICO,QT_CONC_RVPDEF,QT_CONC_RVSOCIAL_RF,QT_CONC_RVOUTROS
0,Centro-Oeste,1,5157,4546,2771,17,1805,96
1,Centro-Oeste,2,47,1,1,1,11,33
2,Nordeste,1,13393,12359,7488,72,5266,216
3,Nordeste,2,356,50,9,0,39,306
4,Norte,1,2556,2185,1414,28,1260,5
5,Norte,2,4,3,1,0,0,0
6,Sudeste,1,16987,12988,7703,123,6055,368
7,Sudeste,2,171,12,106,0,49,12
8,Sul,1,7474,6682,1388,31,2413,145
9,Sul,2,12,9,1,1,1,0


#Genero

In [206]:
df_qtd_ing_regiao_genero = df.groupby(['NO_REGIAO']).agg( QT_ING_MASC = ('QT_ING_MASC', 'sum'),
                                                            QT_ING_FEM = ('QT_ING_FEM', 'sum'),
                                                            ).reset_index()
#df_qtd_ing_regiao_genero

In [207]:
plot_cor(df_qtd_ing_regiao_genero, 'NO_REGIAO', ['QT_ING_MASC', 'QT_ING_FEM'], 'Número Totais de Ingressantes por Região' )

In [210]:
df_qtd_conc_regiao_genero = df.groupby(['NO_REGIAO']).agg( QT_CONC_MASC = ('QT_CONC_MASC', 'sum'),
                                                            QT_CONC_FEM = ('QT_CONC_FEM', 'sum'),
                                                            ).reset_index()
df_qtd_conc_regiao_genero

,NO_REGIAO,QT_CONC_MASC,QT_CONC_FEM
0,Centro-Oeste,45069,72949
1,Nordeste,91899,140775
2,Norte,32882,53882
3,Sudeste,255622,368532
4,Sul,86441,130334


In [211]:
plot_cor(df_qtd_conc_regiao_genero, 'NO_REGIAO', ['QT_CONC_MASC', 'QT_CONC_FEM'], 'Número Totais de Concluintes por Região' )

In [ ]:
df_qtd_ing_regiao_genero.QT_ING_MASC + df_qtd_ing_regiao_genero.QT_ING_FEM

0     345041
1     719867
2     304589
3    1727141
4     667309
dtype: int64

#UNIVAP

In [196]:
univap = df.groupby('CO_IES').agg( QT_ING_MASC = ('QT_ING_MASC', 'sum'),
                                                            QT_ING_FEM = ('QT_ING_FEM', 'sum'),
                                                            ).reset_index()
univap.loc[univap.CO_IES == 275]


,CO_IES,QT_ING_MASC,QT_ING_FEM
131,275,431,688


In [197]:
univap = df.groupby('CO_IES').agg(QT_ING_BRANCA = ('QT_ING_BRANCA','sum'),
                                                            QT_ING_PRETA = ('QT_ING_PRETA','sum'),
                                                            QT_ING_PARDA = ('QT_ING_PARDA', 'sum'),
                                                            QT_ING_AMARELA = ('QT_ING_AMARELA', 'sum'),
                                                            QT_ING_INDIGENA = ('QT_ING_INDIGENA', 'sum'),
                                                            QT_ING_CORND = ('QT_ING_CORND', 'sum'),
                                                            ).reset_index()
univap.loc[univap.CO_IES == 275]

,CO_IES,QT_ING_BRANCA,QT_ING_PRETA,QT_ING_PARDA,QT_ING_AMARELA,QT_ING_INDIGENA,QT_ING_CORND
131,275,445,14,72,7,3,578
